In [1]:
import pandas as pd
import pylibjpeg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import cv2
import os
from os import listdir
import re
import gc
import random
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.auto import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)
import zipfile
from scipy import ndimage
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from PIL import Image
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from kaggle_volclassif.utils import interpolate_volume
from skimage import exposure

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
df_test_pred = pd.read_csv('../input/rsna-2022-spine-fracture-detection-metadata/train_segmented.csv')
df_test_pred

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.10001,1,512,512,0.625,-52.308,-27.712,7.282,0.005208,0.015055,0.003290,0.000895,0.001614,0.004632,0.002570
1,1.2.826.0.1.3680043.10001,2,512,512,0.625,-52.308,-27.712,6.657,0.005675,0.015945,0.002117,0.000562,0.001265,0.005648,0.004136
2,1.2.826.0.1.3680043.10001,3,512,512,0.625,-52.308,-27.712,6.032,0.006624,0.030292,0.003079,0.000677,0.000882,0.002238,0.002877
3,1.2.826.0.1.3680043.10001,4,512,512,0.625,-52.308,-27.712,5.407,0.006572,0.022294,0.003103,0.000975,0.001164,0.002434,0.002715
4,1.2.826.0.1.3680043.10001,5,512,512,0.625,-52.308,-27.712,4.782,0.004945,0.022388,0.003236,0.000826,0.000662,0.002304,0.006849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711596,1.2.826.0.1.3680043.9997,251,512,512,0.625,-55.200,-24.600,-187.750,0.000894,0.000706,0.000682,0.000664,0.000701,0.000529,0.000845
711597,1.2.826.0.1.3680043.9997,252,512,512,0.625,-55.200,-24.600,-188.375,0.000849,0.000754,0.000835,0.000745,0.000791,0.000630,0.000547
711598,1.2.826.0.1.3680043.9997,253,512,512,0.625,-55.200,-24.600,-189.000,0.000715,0.000750,0.000688,0.000460,0.000554,0.000774,0.000704
711599,1.2.826.0.1.3680043.9997,254,512,512,0.625,-55.200,-24.600,-189.625,0.000820,0.001236,0.001794,0.001262,0.001232,0.001555,0.000754
